In [1]:
import sys
sys.path.append('/Users/victor/Documents/code/nba_api')

import re
import pandas as pd
import numpy as np

from src.espn_scraper import espn_player_scraper
from sqlalchemy import create_engine, Integer
from nba_py import player

## Scrape and Load to DB

In [2]:
# Initialize the db. Relatively small dataset, so we'll use SQLite
engine = create_engine("sqlite:///db/nba.db", echo=False)

teams_overview_url = "http://www.espn.com/nba/players"
scraper = espn_player_scraper()
espn_player_list = scraper.scrape_all_players(teams_overview_url)

100%|██████████| 30/30 [00:22<00:00,  1.30it/s]


In [3]:
# Load to DB
with engine.connect() as conn:
    espn_player_list.to_sql('espn_players', conn, dtype={"espn_player_id":Integer}, if_exists="replace")

## Getting player performance data from NBA API
Here we will use nba_py by seemethere, a Python wrapper for the unpublished offical NBA API.

The official NBA API uses a different set of Player IDs, so we will have to join ESPN and NBA data by cross referencing player names. Also, the NBA API returns some players in the NBA Delevelopment league, which we are not interested in. We will thus use a left join on the ESPN table.

In [4]:
# Loading back the data
with engine.connect() as conn:
    espn_player_list = pd.read_sql('espn_players', conn)

# Getting players for the current season
nba_player_list = player.PlayerList().info()

# Convert upper case letter to upper case letter
nba_player_list.columns = [col.lower() for col in nba_player_list.columns]

# Check if number of players are the same
print("NBA roster has {} players".format(nba_player_list.shape[0]))
print("ESPN roster has {} players".format(espn_player_list.shape[0]))

NBA roster has 544 players
ESPN roster has 498 players


In [5]:
with engine.connect() as conn:
    nba_player_list.to_sql("nba_players", conn, dtype={'person_id':Integer}, if_exists="replace", index=False)

In [6]:
nba_player_list.head()

,person_id,display_last_comma_first,display_first_last,rosterstatus,from_year,to_year,playercode,team_id,team_city,team_name,team_abbreviation,team_code,games_played_flag,otherleague_experience_ch
0,203518,"Abrines, Alex",Alex Abrines,0,2016,2018,alex_abrines,0,,,,,Y,00
1,203112,"Acy, Quincy",Quincy Acy,0,2012,2018,quincy_acy,0,,,,,Y,11
2,1629121,"Adams, Jaylen",Jaylen Adams,1,2018,2018,jaylen_adams,1610612737,Atlanta,Hawks,ATL,hawks,Y,11
3,203500,"Adams, Steven",Steven Adams,1,2013,2018,steven_adams,1610612760,Oklahoma City,Thunder,OKC,thunder,Y,00
4,1628389,"Adebayo, Bam",Bam Adebayo,1,2017,2018,bam_adebayo,1610612748,Miami,Heat,MIA,heat,Y,00


In [7]:
# Selecting columns of interest
nba_player_list = nba_player_list[['person_id','display_first_last']]
nba_player_list.head()

,person_id,display_first_last
0,203518,Alex Abrines
1,203112,Quincy Acy
2,1629121,Jaylen Adams
3,203500,Steven Adams
4,1628389,Bam Adebayo


In [8]:
espn_player_list.head()

,index,name,position,espn_player_id,url
0,0,Aron Baynes,C,2968439,http://www.espn.com/nba/player/_/id/2968439/ar...
1,1,Jaylen Brown,SG,3917376,http://www.espn.com/nba/player/_/id/3917376/ja...
2,2,PJ Dozier,PG,3923250,http://www.espn.com/nba/player/_/id/3923250/pj...
3,3,Jonathan Gibson,PG,2234666,http://www.espn.com/nba/player/_/id/2234666/jo...
4,4,Gordon Hayward,SF,4249,http://www.espn.com/nba/player/_/id/4249/gordo...


### Cleaning before join

In [9]:
# Cleaning the data a bit for joins
# Remove all special characters, convert to lower case
def sanitize_name(name_str):
    sanitized = name_str.lower()
    sanitized = sanitized.replace('-', ' ')
    
    to_remove = [".", "'", "jr", 'sr', 'iii']
    for pattern in to_remove:
        sanitized = sanitized.replace(pattern, '')
        
    # Remove trailing space
    sanitized = re.sub(re.compile("\s*$"), '', sanitized)
        
    return sanitized

In [10]:
# Getting a list of ESPN player names not in NBA player name list
espn_player_list[~espn_player_list.name.isin(nba_player_list.display_first_last)].head()

,index,name,position,espn_player_id,url
37,37,Billy Garrett Jr.,G,3059356,http://www.espn.com/nba/player/_/id/3059356/bi...
158,158,Cody Demps,None,4028211,http://www.espn.com/nba/player/_/id/4028211/co...
166,166,Frank Mason III,PG,3059315,http://www.espn.com/nba/player/_/id/3059315/fr...
178,178,Walt Lemon Jr.,PG,2528586,http://www.espn.com/nba/player/_/id/2528586/wa...
213,213,Sviatoslav Mykhailiuk,SG,3133602,http://www.espn.com/nba/player/_/id/3133602/sv...


In [11]:
# Sanitize both dfs
nba_player_list.display_first_last = nba_player_list.display_first_last.astype('str')
espn_player_list.name = espn_player_list.name.astype('str')

espn_player_list.name = espn_player_list.name.apply(sanitize_name)
nba_player_list.display_first_last = nba_player_list.display_first_last.apply(sanitize_name)

In [12]:
# Handle a few special cases where names are different across two 
espn_player_list.loc[espn_player_list.espn_player_id == 1713, 'name'] = 'nene'
espn_player_list.loc[espn_player_list.espn_player_id == 4017839, 'name'] = 'juancho hernangomez'
espn_player_list.loc[espn_player_list.espn_player_id == 3056247, 'name'] = 'kendrick nunn'
espn_player_list.loc[espn_player_list.espn_player_id == 2528586, 'name'] = 'walter lemon'
espn_player_list.loc[espn_player_list.espn_player_id == 3133602, 'name'] = 'svi mykhailiuk'

In [13]:
merged_df = espn_player_list.join(nba_player_list.set_index("display_first_last", drop=True),
                                    on='name')

In [14]:
# Check for rows that didn't join correctly
merged_df[merged_df.isnull().any(axis=1)]

,index,name,position,espn_player_id,url,person_id
158,158,cody demps,None,4028211,http://www.espn.com/nba/player/_/id/4028211/co...,NaN
376,376,kendrick nunn,None,3056247,http://www.espn.com/nba/player/_/id/3056247/ke...,1629134.0


In [19]:
merged_df[merged_df.position.isnull()]

,index,name,position,espn_player_id,url,person_id
158,158,cody demps,None,4028211,http://www.espn.com/nba/player/_/id/4028211/co...,NaN
376,376,kendrick nunn,None,3056247,http://www.espn.com/nba/player/_/id/3056247/ke...,1629134.0


In [27]:
# Kendrick Nunn and Cody Demps just got traded and 
# has not played any games in his career, so we're dropping them

merged_df = merged_df.dropna()
merged_df = merged_df.rename(columns={"person_id":"nba_id"})
merged_df.shape

(496, 6)

In [29]:
# Load to DB
with engine.connect() as conn:
    merged_df.to_sql("players", conn, dtype={'nba_id':Integer, 'espn_player_id':Integer}, index=False, if_exists='replace')

In [30]:
merged_df.head()

,index,name,position,espn_player_id,url,nba_id
0,0,aron baynes,C,2968439,http://www.espn.com/nba/player/_/id/2968439/ar...,203382.0
1,1,jaylen brown,SG,3917376,http://www.espn.com/nba/player/_/id/3917376/ja...,1627759.0
2,2,pj dozier,PG,3923250,http://www.espn.com/nba/player/_/id/3923250/pj...,1628408.0
3,3,jonathan gibson,PG,2234666,http://www.espn.com/nba/player/_/id/2234666/jo...,1626780.0
4,4,gordon hayward,SF,4249,http://www.espn.com/nba/player/_/id/4249/gordo...,202330.0


## Loading NBA Player Stat into DB